In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

# преобразует строковые параметры в числа 
# Exampe: "Hello" "Hello" "world" [0,0,1]
class MultiColumnLabelEncoder:
    def __init__(self, columns=None):
        self.columns = columns  # array of column names to encode

    def fit(self, X, y=None):
        return self  # not relevant here

    def transform(self, X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = preprocessing.LabelEncoder().fit_transform(output[col])
        else:
            for col_name, col in output.iteritems():
                output[col_name] = preprocessing.LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)

In [2]:
# чтение файла с объектами
df = pd.read_excel('1.xlsx')

In [3]:

df

,ID,NAME,PARENT_ID,LOGIN,COL_754,COL_755,COL_756,COL_757,COL_758,COL_759,...,COL_775,COL_781,COL_782,COL_2156,COL_2463,COL_3163,COL_3243,COL_3363,COL_3468,COL_103506
0,NaN,NaN,NaN,NaN,Назначение,Форма собственности,Год постройки,Год реконструкции,Серия проекта,Количество этажей,...,Очередность уборки кровли,Материал кровли,UNOM,Вид социального объекта,Тип жилищного фонда,Статус МКД,Статус управления МКД,Количество грузовых лифтов,Причина Изменения Статуса МКД,Категория МКД
1,1500923.0,"Дом по адресу Красковская ул., д.121А",NaN,WS,NaN,NaN,1956,NaN,2048798,1,...,NaN,22289204,49928,NaN,22741963,58761330,NaN,0,NaN,NaN
2,1500933.0,"Дом по адресу Байкальская ул., д.18, к.2",NaN,WS,NaN,NaN,2005,NaN,2048798,18,...,NaN,22289201,64148,NaN,42875644,58761330,45063584,0,NaN,NaN
3,1500943.0,"Дом по адресу Перовская ул., д.48",NaN,WS,NaN,NaN,1955,NaN,2048912,4,...,22289162,22289204,19485,NaN,42875644,58761330,45063584,0,NaN,NaN
4,1500951.0,"Дом по адресу Парковая 3-я ул., д.39, к.3",NaN,404070001,NaN,NaN,1960,NaN,2048912,5,...,22289162,22289201,18533,NaN,42875644,58761330,45063584,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18428,544441568.0,"Дом по адресу Салтыковская ул., д.6/1, к.5",NaN,operator2,Нет техпаспорта МосгорБТИ,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,6003997,NaN,42875644,58761330,45063584,NaN,NaN,NaN
18429,545483918.0,"Дом по адресу Амурская ул., д.2А, к.1",NaN,operator2,Нет техпаспорта МосгорБТИ,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5259334,NaN,42875644,58761330,45063584,NaN,NaN,NaN
18430,545483942.0,"Дом по адресу Амурская ул., д.2А, к.2",NaN,operator2,Нет техпаспорта МосгорБТИ,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5259335,NaN,42875644,58761330,45063584,NaN,NaN,NaN
18431,545483949.0,"Дом по адресу Амурская ул., д.2А, к.3",NaN,operator2,Нет техпаспорта МосгорБТИ,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5259332,NaN,42875644,58761330,45063584,NaN,NaN,NaN


In [4]:
# чистка
df.drop([
    'LOGIN', 'COL_755',
    'COL_754', 'COL_765', 'COL_2156',
    'COL_103506','PARENT_ID',
], axis=1, inplace=True)

In [5]:
df = df[1:]

In [6]:
# преобразуем колонку в единый вид
def get_word(x):
    x = x.lower().split(' ')
    res = ' '
    if 'распоряжение' in x:
        res = 'распоряжение'
    elif 'обращение' in x:
        res = 'обращение'
    elif 'письмо' in x:
        res = 'письмо'
    return res

# pd.options.mode.chained_assignment = None 
df.loc[:, 'COL_3468'] = df['COL_3468'].astype('str').apply(lambda x: get_word(x))

In [7]:
# Преобразуем в числовые параметры
df = MultiColumnLabelEncoder(columns=[
    'COL_757', 'COL_763', 'COL_764',
    'COL_766', 'COL_767', 'COL_772',
    'COL_775', 'COL_781', 'COL_2463',
    'COL_3468', 'COL_762',
]).fit_transform(df)

In [8]:
# Чистка
df.fillna({
    'COL_758': 0, 'COL_759': 0, 'COL_760': 0,
    'COL_761': 0, 'COL_770': 0, 'COL_771': 0,
    'COL_2463': 0, 'COL_3243': 0, 'COL_3363': 0,
}, inplace=True)
# Выделяем связь с инцидентами
df.rename({
    'COL_782':'UNOM',
}, axis=1, inplace=True)

In [9]:
df

,ID,NAME,COL_756,COL_757,COL_758,COL_759,COL_760,COL_761,COL_762,COL_763,...,COL_771,COL_772,COL_775,COL_781,UNOM,COL_2463,COL_3163,COL_3243,COL_3363,COL_3468
1,1500923.0,"Дом по адресу Красковская ул., д.121А",1956,1,2048798,1,2,1,5319,3035,...,0,0,2,2,49928,1,58761330,0,0,0
2,1500933.0,"Дом по адресу Байкальская ул., д.18, к.2",2005,1,2048798,18,2,148,5523,3131,...,2,7,2,0,64148,3,58761330,45063584,0,0
3,1500943.0,"Дом по адресу Перовская ул., д.48",1955,1,2048912,4,2,20,584,124,...,0,0,0,2,19485,3,58761330,45063584,0,0
4,1500951.0,"Дом по адресу Парковая 3-я ул., д.39, к.3",1960,1,2048912,5,3,60,2181,1145,...,0,0,0,0,18533,3,58761330,45063584,0,0
5,1500954.0,"Дом по адресу Свободный просп., д.11, к.2",1973,1,2048755,9,12,431,1796,908,...,12,0,2,0,22830,3,58761330,45063584,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18428,544441568.0,"Дом по адресу Салтыковская ул., д.6/1, к.5",NaN,1,0,0,0,0,6022,3381,...,0,16,2,10,6003997,3,58761330,45063584,0,0
18429,545483918.0,"Дом по адресу Амурская ул., д.2А, к.1",NaN,1,0,0,0,0,6022,3381,...,0,16,2,10,5259334,3,58761330,45063584,0,0
18430,545483942.0,"Дом по адресу Амурская ул., д.2А, к.2",NaN,1,0,0,0,0,6022,3381,...,0,16,2,10,5259335,3,58761330,45063584,0,0
18431,545483949.0,"Дом по адресу Амурская ул., д.2А, к.3",NaN,1,0,0,0,0,6022,3381,...,0,16,2,10,5259332,3,58761330,45063584,0,0


In [10]:
# df = df.join() COL782 - UNOM
# открываем файл с инцидентами лист 1
work_types = pd.read_excel('2.xlsx', sheet_name='Result 1')
work_types

,Наименование,Источник,Дата создания во внешней системе,Дата закрытия,Округ,Адрес,unom,Дата и время завершения события во
0,Открыт колодец,MVK,2022-08-30 14:41:53.000000,2022-08-30 15:40:26.565000,ВАО,внутригородская территория муниципальный округ...,2115191,NaN
1,Отклонение ГВС ниже нормы ночью (мониторинг),ASUPR,2022-02-20 06:24:07.000000,2022-04-04 02:43:22.101000,ВАО,внутригородская территория муниципальный округ...,2404461,NaN
2,Отклонение ГВС ниже нормы днем (мониторинг),ASUPR,2022-03-07 21:04:01.000000,2022-03-09 09:33:24.894000,ВАО,внутригородская территория муниципальный округ...,2404461,NaN
3,Температура ГВС ниже нормы,ASUPR,2022-03-05 22:13:33.000000,2022-03-05 19:14:00.190000,ВАО,внутригородская территория муниципальный округ...,2404461,NaN
4,Температура ГВС ниже нормы,ASUPR,2022-02-18 21:43:54.000000,2022-02-18 18:44:14.682000,ВАО,внутригородская территория муниципальный округ...,2404461,NaN
...,...,...,...,...,...,...,...,...
1048570,Отсутствие освещения в местах общего пользования,EDC,2022-11-23 11:56:32.000000,2022-11-23 12:22:34.591000,ВАО,внутригородская территория муниципальный округ...,4086,2022-11-23 12:22:34.045000
1048571,Температура в квартире ниже нормативной,EDC,2022-11-23 15:37:33.000000,2022-11-23 16:57:53.149000,ВАО,внутригородская территория муниципальный округ...,4086,2022-11-23 16:57:53.149000
1048572,Недогрев ГВС,EDC,2022-11-24 08:01:52.000000,2022-11-24 09:08:46.374000,ВАО,внутригородская территория муниципальный округ...,4086,2022-11-24 09:08:46.374000
1048573,Отсутствие освещения в местах общего пользования,EDC,2022-11-24 11:56:22.000000,2022-11-24 13:56:41.609000,ВАО,внутригородская территория муниципальный округ...,4086,2022-11-24 13:56:41.609000


In [11]:
# открываем файл с инцидентами лист 2
work_types2 = pd.read_excel('2.xlsx', sheet_name='Result 2')
work_types2

,Наименование,Источник,Дата создания во внешней системе,Дата закрытия,Округ,Адрес,unom,Дата и время завершения события во
0,Отсутствие освещения в местах общего пользования,EDC,2022-11-24 13:37:48.000000,2022-11-24 16:34:46.849000,ВАО,внутригородская территория муниципальный округ...,4086,2022-11-24 16:34:46.849000
1,Отсутствие освещения в местах общего пользования,EDC,2022-11-24 16:12:45.000000,2022-11-24 16:34:10.370000,ВАО,внутригородская территория муниципальный округ...,4086,2022-11-24 16:34:09.752000
2,Отсутствие освещения в местах общего пользования,EDC,2022-11-24 16:26:15.000000,2022-11-24 17:08:52.549000,ВАО,внутригородская территория муниципальный округ...,4086,2022-11-24 17:08:52.548000
3,Отсутствие освещения в местах общего пользования,EDC,2022-11-24 16:39:24.000000,2022-11-24 17:09:27.741000,ВАО,внутригородская территория муниципальный округ...,4086,2022-11-24 17:09:27.491000
4,Температура в квартире ниже нормативной,EDC,2022-11-24 16:42:50.000000,2022-11-24 17:09:45.926000,ВАО,внутригородская территория муниципальный округ...,4086,2022-11-24 17:09:43.627000
...,...,...,...,...,...,...,...,...
5301,Засор канализации,EDC,2022-10-28 08:26:59.000000,2022-10-28 08:45:46.634000,ВАО,внутригородская территория муниципальный округ...,14257,2022-10-28 08:45:46.634000
5302,Недогрев ГВС,EDC,2022-11-02 17:51:03.000000,2022-11-02 19:42:59.116000,ВАО,внутригородская территория муниципальный округ...,14257,2022-11-02 19:42:59.116000
5303,Недогрев ГВС,EDC,2022-11-03 10:19:13.000000,2022-11-03 10:47:10.487000,ВАО,внутригородская территория муниципальный округ...,14257,2022-11-03 10:47:10.487000
5304,Недогрев ГВС,EDC,2022-11-03 10:20:33.000000,2022-11-03 10:47:15.575000,ВАО,внутригородская территория муниципальный округ...,14257,2022-11-03 10:47:15.575000


In [12]:
# Матчим листы и приводим к единому виду
work_types = pd.concat([work_types, work_types2])
work_types['Дата создания во внешней системе'].fillna('01-01-1970 ', inplace=True)
work_types['Дата закрытия'].fillna('01-01-1970 ', inplace=True)
work_types['Дата создания во внешней системе'] = (work_types['Дата создания во внешней системе']
                             .apply(
    lambda x: x.split(' ')[0].replace('-', '.'))
)
work_types['Дата закрытия'] = (work_types['Дата закрытия']
                             .apply(
    lambda x: x.split(' ')[0].replace('-', '.'))
)
work_types.rename({
    'Наименование':'WORK_NAME',
    'Дата создания во внешней системе':'CREATE_DATE',
    'Дата закрытия':'CLOSE_DATE',
    'Источник':'SOURCE',
    'unom':'UNOM',
}, axis=1, inplace=True)
work_types = work_types[['WORK_NAME', 'CREATE_DATE', 'CLOSE_DATE', 'SOURCE', 'UNOM']]


work_types

,WORK_NAME,CREATE_DATE,CLOSE_DATE,SOURCE,UNOM
0,Открыт колодец,2022.08.30,2022.08.30,MVK,2115191
1,Отклонение ГВС ниже нормы ночью (мониторинг),2022.02.20,2022.04.04,ASUPR,2404461
2,Отклонение ГВС ниже нормы днем (мониторинг),2022.03.07,2022.03.09,ASUPR,2404461
3,Температура ГВС ниже нормы,2022.03.05,2022.03.05,ASUPR,2404461
4,Температура ГВС ниже нормы,2022.02.18,2022.02.18,ASUPR,2404461
...,...,...,...,...,...
5301,Засор канализации,2022.10.28,2022.10.28,EDC,14257
5302,Недогрев ГВС,2022.11.02,2022.11.02,EDC,14257
5303,Недогрев ГВС,2022.11.03,2022.11.03,EDC,14257
5304,Недогрев ГВС,2022.11.03,2022.11.03,EDC,14257


In [13]:
# окрываем файл с инцидентами, действем по аналогии с предыдушим, чистим и приводим к единому виду
work_types3 = pd.read_excel('3.xlsx')
work_types3['SOURCE'] = 'DST'

work_types3.rename({
    'FACT_DATE_START':'CREATE_DATE',
    'FACT_DATE_END':'CLOSE_DATE',
}, axis=1, inplace=True)
work_types3 = work_types3[['WORK_NAME', 'CREATE_DATE', 'CLOSE_DATE', 'SOURCE', 'UNOM']]

In [14]:
# приводим даты к общему виду из файла3 к файлу2 (именно в этот файл т.к. он меньше)
def reverse_date(x):
    date = x.split('.')
    return date[2] + '.' + date[1] + '.' + date[0]

work_types3['CREATE_DATE'] = (work_types3['CREATE_DATE']
                             .apply(
    lambda x: reverse_date(x))
)
work_types3['CLOSE_DATE'] = (work_types3['CLOSE_DATE']
                             .apply(
    lambda x: reverse_date(x))
)
work_types3

,WORK_NAME,CREATE_DATE,CLOSE_DATE,SOURCE,UNOM
0,замена лифтового оборудования,2022.06.27,2022.10.14,DST,20151
1,замена лифтового оборудования,2022.09.30,2022.10.21,DST,20151
2,ремонт внутридомовых инженерных систем водоотв...,2022.02.01,2022.05.25,DST,11466
3,ремонт внутридомовых инженерных систем теплосн...,2022.05.07,2022.08.03,DST,11466
4,ремонт внутридомовых инженерных систем холодно...,2022.02.01,2022.06.27,DST,11466
...,...,...,...,...,...
1742,ремонт внутридомовых инженерных систем теплосн...,2022.03.07,2022.08.11,DST,9177
1743,"ремонт подвальных помещений, относящихся к общ...",2022.02.04,2022.06.17,DST,9177
1744,ремонт внутридомовых инженерных систем водоотв...,2022.02.04,2022.08.04,DST,9177
1745,ремонт внутреннего водостока,2022.10.10,2022.11.30,DST,12178


In [15]:
# объединяем все инциденты
work_types = pd.concat([work_types, work_types3])
work_types

,WORK_NAME,CREATE_DATE,CLOSE_DATE,SOURCE,UNOM
0,Открыт колодец,2022.08.30,2022.08.30,MVK,2115191
1,Отклонение ГВС ниже нормы ночью (мониторинг),2022.02.20,2022.04.04,ASUPR,2404461
2,Отклонение ГВС ниже нормы днем (мониторинг),2022.03.07,2022.03.09,ASUPR,2404461
3,Температура ГВС ниже нормы,2022.03.05,2022.03.05,ASUPR,2404461
4,Температура ГВС ниже нормы,2022.02.18,2022.02.18,ASUPR,2404461
...,...,...,...,...,...
1742,ремонт внутридомовых инженерных систем теплосн...,2022.03.07,2022.08.11,DST,9177
1743,"ремонт подвальных помещений, относящихся к общ...",2022.02.04,2022.06.17,DST,9177
1744,ремонт внутридомовых инженерных систем водоотв...,2022.02.04,2022.08.04,DST,9177
1745,ремонт внутреннего водостока,2022.10.10,2022.11.30,DST,12178


In [16]:
# приводим все описания инцидентов в lowercase и смотрим какие уникальные
work_types['WORK_NAME'] = work_types['WORK_NAME'].str.lower()
work_types['WORK_NAME'].unique()

array(['открыт колодец', 'отклонение гвс ниже нормы ночью (мониторинг)',
       'отклонение гвс ниже нормы днем (мониторинг)',
       'температура гвс ниже нормы', 'отсутствует циркуляция гвс',
       't1 > max', 'изменение конфигурации успд',
       'расхождение времени пу',
       'критичное отклонение температуры гвс ниже нормы днем (мониторинг)',
       'низкий уровень сигнала gsm', 'нет связи с пу',
       'отсутствуют актуальные суточные значения',
       'отсутствуют актуальные мгновенные значения',
       'критичное отклонение температуры гвс ниже нормы ночью (мониторинг)',
       'нет питания успд', 'открыт шкаф успд', 'нет связи с успд',
       'угроза взрыва',
       'отсутствие освещения в местах общего пользования',
       'загрязнение/замусоренность подъезда',
       'повреждение системы электропроводки/щитового оборудования',
       'повреждение отделочных покрытий пола/стены/ступеней/перил/других элементов',
       'загрязнение воды', 'повреждение элементов входной двер

In [17]:
# определяем словарь и расбиваем признаки на категории в идеале использовать нужно nltk 
alpabet = {
'СК': ['строительный', 'контроль', 'ск'],
'Подъезд': ['подъездов,', 'подъезд', 'инвалид', 'инвалид', 'полос', 'почтовый', 'почтовых',
            'входной двери', 'входная дверь', 'панельных швов', 'панельные швы', 'отделочных', 'отделка',
            'пандус', 'светильник'],
'Окна': ['оконных', 'окна', 'окно'],
'Проведение экспертизы': ['проведение', 'экспертизы', 'проектной', 'документации', 'проведение экспертизы'],
'ПСД': ['проектная', 'проектной', 'документации', 'документация', 'псд'],
'ЦО-М': ['теплоснабжения', 'цо-м', 'теплоснабжение', 'тепло', 'отопление', 'отопления'],
'КАН-М': ['водоотведения','водоотведение', 'канализации', 'канализация', 'трубопроводы', 'кан-м',  
          'воды', 'вода', 'трубопровод', 'колодца', 'подтопление', 'подтопления'],
'ГВС-М': ['горячего', 'горячее', 'водоснабжения', 'водоснабжение', 'гвс-м'],
'ХВС-М': ['холодного', 'холодное', 'водоснабжения', 'водоснабжение', 'хвс-м'],
'Мусоропровод': ['мусоропровода', 'мусоропровод', 'мусор'],
'ПВ': ['пожарного', 'пожарный', 'водопровода', 'водопровод', 'пв'],
'ППАиДУ': ['дым', 'внутридомовой', 'системы', 'дымоудаления', 'противопожарной', 'автоматики', 'ппаиду',
           'пожарной'],
'ЭС': ['электроснабжения', 'проводка', 'проводка', 'эс', 'электро', 'электропроводка', 'электропроводки'],
'ГАЗ': ['газоснабжения', 'газоснабжение', 'газ', 'ковера', 'ковер'],
'Подвал': ['подвальных', 'подвальный', 'подвал'],
'ЦО': [ 'теплоснабжения','тепло', 'стояки', 'цо'],
'КАН': [ 'водоотведения', 'водоотведение', 'канализации', 'канализация', 'кан', 'колодцы', 'колодец', 'засоры', 'засор'],
'ГВС': [ 'горячего', 'горячее', 'горячий', 'водоснабжения', 'водоснабжениие', 'гвс'],
'Фундамент': [ 'фундамента', 'фундамент'],
'ХВС': [ 'холодного', 'водоснабжения', 'водоснабжение', 'хвс'],
'Крыша': [ 'крыши', 'крыша', 'кровля', 'кровли'],
'ВДСК': [ 'водостока','водосток', 'вдск'],
'Балкон': [ 'балконов', 'балкон', 'балконы'],
'Фасад': [ 'фасада', 'фасад', 'фасадов', 'фасады'],
'Разработка и проведение экспертизы ПД, в том числе АН за проведением работ по сохранению ОКН, выявленных ОКН, научное руководство проведением указанных работ в случае проведения работ по КР ОИ в МКД  г. Москвы, являющихся ОКН, выявленными ОКН.':
 ['разработка', 'проведение', 'экспертизы', 'числе', 'проведением', 'работ', 'сохранению', 'окн,', 'выявленных', 'окн,', 'научное', 'руководство', 'проведением', 'указанных', 'работ', 'случае', 'проведения', 'работ', 'москвы,', 'являющихся', 'окн,', 'выявленными', 'окн.', 'разработка и проведение экспертизы пд, в том числе ан за проведением работ по сохранению окн, выявленных окн, научное руководство проведением указанных работ в случае проведения работ по кр ои в мкд  г. москвы, являющихся окн, выявленными окн.'],
'Авторский надзор': ['авторский', 'надзор', 'авторский надзор'],
'ПСД лифты': ['разработка', 'проектной', 'документации', 'лифтам', 'псд лифты'],
'Лифт': ['лифтового', 'оборудования', 'лифт', 'лифты'],
'Телемеханика':['нет связи с пу', 'отсутствуют ',
 'актуальные', 'значения', 'dsa',
 'успд', 'угроза', 'взрыв', 'устройства', 'устройство', 'аномальное', 'значение',
 'связь', 'связи', 'вибрации', 'датчик', 'интегральные', 't1', 'p1', 'p2', 'расхождение', 'расхождения', 'gsm'],
'Освещение': ['освещения', 'освещение']}
print(list(alpabet.keys()).index("КАН"))

16


In [18]:
# types_workin = pd.read_excel('4.xlsx')[1:]
# матчим инцидент с конкретным видом работ из категории выше (чисто тест)
def check_in_type(x, alpabet):
    alpabet.keys()
    scores = {}
    for k, v in alpabet.items():
        for tp in v:
            if not isinstance(x, str) or x == '':
                scores['Другое'] = scores['Другое'] + 1 if scores.get('Другое') else 1
                continue
            if tp in x:
                if scores.get(k):
                    scores[k] += 1
                else:
                    scores[k] = 1
    # print('Очки', scores)
    if scores:
        # return max(scores, key=scores.get)
        res = max(scores, key=scores.get)
        if res == 'Другое':
            return 30
        return list(alpabet.keys()).index(res)
    else:
        return -1
        # return 30

        
# print(check_in_type("", alpabet))
work_types['WORK_CLASS'] = (work_types['WORK_NAME']
                            .apply(lambda x: check_in_type(x, alpabet)))
# work_types[['WORK_CLASS', 'WORK_NAME']]


In [19]:
pd.DataFrame.iteritems = pd.DataFrame.items
# преобразуем в числа
work_types = MultiColumnLabelEncoder(columns=['WORK_CLASS']).fit_transform(work_types)
work_types

,WORK_NAME,CREATE_DATE,CLOSE_DATE,SOURCE,UNOM,WORK_CLASS
0,открыт колодец,2022.08.30,2022.08.30,MVK,2115191,15
1,отклонение гвс ниже нормы ночью (мониторинг),2022.02.20,2022.04.04,ASUPR,2404461,16
2,отклонение гвс ниже нормы днем (мониторинг),2022.03.07,2022.03.09,ASUPR,2404461,16
3,температура гвс ниже нормы,2022.03.05,2022.03.05,ASUPR,2404461,16
4,температура гвс ниже нормы,2022.02.18,2022.02.18,ASUPR,2404461,16
...,...,...,...,...,...,...
1742,ремонт внутридомовых инженерных систем теплосн...,2022.03.07,2022.08.11,DST,9177,4
1743,"ремонт подвальных помещений, относящихся к общ...",2022.02.04,2022.06.17,DST,9177,13
1744,ремонт внутридомовых инженерных систем водоотв...,2022.02.04,2022.08.04,DST,9177,5
1745,ремонт внутреннего водостока,2022.10.10,2022.11.30,DST,12178,19


In [20]:
# чистим инциденты
work_types = work_types[work_types['WORK_NAME'].notna()]
work_types

,WORK_NAME,CREATE_DATE,CLOSE_DATE,SOURCE,UNOM,WORK_CLASS
0,открыт колодец,2022.08.30,2022.08.30,MVK,2115191,15
1,отклонение гвс ниже нормы ночью (мониторинг),2022.02.20,2022.04.04,ASUPR,2404461,16
2,отклонение гвс ниже нормы днем (мониторинг),2022.03.07,2022.03.09,ASUPR,2404461,16
3,температура гвс ниже нормы,2022.03.05,2022.03.05,ASUPR,2404461,16
4,температура гвс ниже нормы,2022.02.18,2022.02.18,ASUPR,2404461,16
...,...,...,...,...,...,...
1742,ремонт внутридомовых инженерных систем теплосн...,2022.03.07,2022.08.11,DST,9177,4
1743,"ремонт подвальных помещений, относящихся к общ...",2022.02.04,2022.06.17,DST,9177,13
1744,ремонт внутридомовых инженерных систем водоотв...,2022.02.04,2022.08.04,DST,9177,5
1745,ремонт внутреннего водостока,2022.10.10,2022.11.30,DST,12178,19


In [21]:
# по хорошему преобразовать дату к числу (int - timestamp) и отсортировать
work_types = MultiColumnLabelEncoder(columns=[
    'WORK_NAME', 'CREATE_DATE', 'CLOSE_DATE',
    'SOURCE',
]).fit_transform(work_types)
work_types

,WORK_NAME,CREATE_DATE,CLOSE_DATE,SOURCE,UNOM,WORK_CLASS
0,186,241,242,7,2115191,15
1,184,50,94,0,2404461,16
2,183,65,68,0,2404461,16
3,300,63,64,0,2404461,16
4,300,48,49,0,2404461,16
...,...,...,...,...,...,...
1742,284,65,223,2,9177,4
1743,291,34,168,2,9177,13
1744,279,34,216,2,9177,5
1745,278,282,334,2,12178,19


In [22]:
# Навсякий случай приводим к int
df['UNOM'] = df['UNOM'].astype(int)
work_types['UNOM'] = work_types['UNOM'].astype(int)
# объединяем с таблицей объктов
big_df = df.merge(work_types, on='UNOM', how='left')


In [23]:
big_df

,ID,NAME,COL_756,COL_757,COL_758,COL_759,COL_760,COL_761,COL_762,COL_763,...,COL_2463,COL_3163,COL_3243,COL_3363,COL_3468,WORK_NAME,CREATE_DATE,CLOSE_DATE,SOURCE,WORK_CLASS
0,1500923.0,"Дом по адресу Красковская ул., д.121А",1956,1,2048798,1,2,1,5319,3035,...,1,58761330,0,0,0,NaN,NaN,NaN,NaN,NaN
1,1500933.0,"Дом по адресу Байкальская ул., д.18, к.2",2005,1,2048798,18,2,148,5523,3131,...,3,58761330,45063584,0,0,298.0,6.0,7.0,3.0,0.0
2,1500933.0,"Дом по адресу Байкальская ул., д.18, к.2",2005,1,2048798,18,2,148,5523,3131,...,3,58761330,45063584,0,0,298.0,6.0,7.0,3.0,0.0
3,1500933.0,"Дом по адресу Байкальская ул., д.18, к.2",2005,1,2048798,18,2,148,5523,3131,...,3,58761330,45063584,0,0,104.0,5.0,7.0,3.0,15.0
4,1500933.0,"Дом по адресу Байкальская ул., д.18, к.2",2005,1,2048798,18,2,148,5523,3131,...,3,58761330,45063584,0,0,298.0,6.0,7.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066468,544441568.0,"Дом по адресу Салтыковская ул., д.6/1, к.5",NaN,1,0,0,0,0,6022,3381,...,3,58761330,45063584,0,0,NaN,NaN,NaN,NaN,NaN
1066469,545483918.0,"Дом по адресу Амурская ул., д.2А, к.1",NaN,1,0,0,0,0,6022,3381,...,3,58761330,45063584,0,0,NaN,NaN,NaN,NaN,NaN
1066470,545483942.0,"Дом по адресу Амурская ул., д.2А, к.2",NaN,1,0,0,0,0,6022,3381,...,3,58761330,45063584,0,0,NaN,NaN,NaN,NaN,NaN
1066471,545483949.0,"Дом по адресу Амурская ул., д.2А, к.3",NaN,1,0,0,0,0,6022,3381,...,3,58761330,45063584,0,0,NaN,NaN,NaN,NaN,NaN


In [24]:
# снова чистим, если нет класса
big_df = big_df[big_df['WORK_CLASS'].notna()]
big_df

,ID,NAME,COL_756,COL_757,COL_758,COL_759,COL_760,COL_761,COL_762,COL_763,...,COL_2463,COL_3163,COL_3243,COL_3363,COL_3468,WORK_NAME,CREATE_DATE,CLOSE_DATE,SOURCE,WORK_CLASS
1,1500933.0,"Дом по адресу Байкальская ул., д.18, к.2",2005,1,2048798,18,2,148,5523,3131,...,3,58761330,45063584,0,0,298.0,6.0,7.0,3.0,0.0
2,1500933.0,"Дом по адресу Байкальская ул., д.18, к.2",2005,1,2048798,18,2,148,5523,3131,...,3,58761330,45063584,0,0,298.0,6.0,7.0,3.0,0.0
3,1500933.0,"Дом по адресу Байкальская ул., д.18, к.2",2005,1,2048798,18,2,148,5523,3131,...,3,58761330,45063584,0,0,104.0,5.0,7.0,3.0,15.0
4,1500933.0,"Дом по адресу Байкальская ул., д.18, к.2",2005,1,2048798,18,2,148,5523,3131,...,3,58761330,45063584,0,0,298.0,6.0,7.0,3.0,0.0
5,1500933.0,"Дом по адресу Байкальская ул., д.18, к.2",2005,1,2048798,18,2,148,5523,3131,...,3,58761330,45063584,0,0,298.0,6.0,7.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066431,464022234.0,"Дом по адресу Тюменский пр., д.3, к.5",NaN,1,0,0,0,0,6022,3381,...,3,58761330,45063584,0,0,115.0,222.0,255.0,7.0,5.0
1066437,464781953.0,"Дом по адресу Парковая 15-я ул., д.42, к.5",NaN,1,0,0,0,0,6022,3381,...,3,58761330,45063584,0,0,108.0,89.0,174.0,7.0,15.0
1066438,464781953.0,"Дом по адресу Парковая 15-я ул., д.42, к.5",NaN,1,0,0,0,0,6022,3381,...,3,58761330,45063584,0,0,108.0,87.0,174.0,7.0,15.0
1066439,464781953.0,"Дом по адресу Парковая 15-я ул., д.42, к.5",NaN,1,0,0,0,0,6022,3381,...,3,58761330,45063584,0,0,312.0,104.0,209.0,7.0,5.0


In [25]:
big_df = MultiColumnLabelEncoder(columns=['COL_756', 'COL_769']).fit_transform(big_df)
big_df['ID'] = big_df['ID'].astype(int)

In [26]:
# готовим веса (по конкретному адресу) по признаку unom:
# 
"""
id      name            unom        create_date
1       Парковая 15     123123      01.01.23
1       Парковая 15     123124      02.01.23
1       Парковая 15     321321      03.01.23

Итог:
id      name            unom        create_date
1       Парковая 15     123123          0
1       Парковая 15     246246          1
1       Парковая 15     567567          2

Можно так: преобразуем unom и дату 
id      name            unom        create_date
1       Парковая 15     0          0
1       Парковая 15     1          1
1       Парковая 15     2          2

На выходе:
id      name            unom        create_date
1       Парковая 15     0          0
1       Парковая 15     1          1
1       Парковая 15     3          2

Конкретный адрес с максимальной суммой unom сохраняем в бд, а обучаем на исторических
Например:
Сохраняем в БД
id      name            unom        create_date
1       Парковая 15     3          2
Обучаем на интеграционной таблице
id      name            unom        create_date
1       Парковая 15     0          0
1       Парковая 15     1          1
1       Парковая 15     3          2

"""

'\nid      name            unom        create_date\n1       Парковая 15     123123      01.01.23\n1       Парковая 15     123124      02.01.23\n1       Парковая 15     321321      03.01.23\n\nИтог:\nid      name            unom        create_date\n1       Парковая 15     123123          0\n1       Парковая 15     246246          1\n1       Парковая 15     567567          2\n\nМожно так: преобразуем unom и дату \nid      name            unom        create_date\n1       Парковая 15     0          0\n1       Парковая 15     1          1\n1       Парковая 15     2          2\n\nНа выходе:\nid      name            unom        create_date\n1       Парковая 15     0          0\n1       Парковая 15     1          1\n1       Парковая 15     3          2\n\nКонкретный адрес с максимальной суммой unom сохраняем в бд, а обучаем на исторических\nНапример:\nСохраняем в БД\nid      name            unom        create_date\n1       Парковая 15     3          2\nОбучаем на интеграционной таблице\nid    

In [27]:
builds = big_df['NAME'].unique().tolist()
sum_df = pd.DataFrame()
builds_df = None
for build in builds:
    builds_df = big_df[big_df['NAME'] == build].sort_values(by='CREATE_DATE')
    builds_df = builds_df.reset_index()
    last = 0
    for i, row in builds_df.iterrows(): 
        if i == 0:
            last = builds_df.at[i,'UNOM']
            continue
        last += builds_df.at[i,'UNOM']
        builds_df.at[i,'UNOM'] += last
    sum_df = pd.concat([sum_df, builds_df])
    
sum_df

,index,ID,NAME,COL_756,COL_757,COL_758,COL_759,COL_760,COL_761,COL_762,...,COL_2463,COL_3163,COL_3243,COL_3363,COL_3468,WORK_NAME,CREATE_DATE,CLOSE_DATE,SOURCE,WORK_CLASS
0,76,1500933,"Дом по адресу Байкальская ул., д.18, к.2",124,1,2048798,18,2,148,5523,...,3,58761330,45063584,0,0,197.0,0.0,1.0,3.0,25.0
1,77,1500933,"Дом по адресу Байкальская ул., д.18, к.2",124,1,2048798,18,2,148,5523,...,3,58761330,45063584,0,0,111.0,0.0,1.0,3.0,23.0
2,85,1500933,"Дом по адресу Байкальская ул., д.18, к.2",124,1,2048798,18,2,148,5523,...,3,58761330,45063584,0,0,140.0,0.0,1.0,3.0,23.0
3,37,1500933,"Дом по адресу Байкальская ул., д.18, к.2",124,1,2048798,18,2,148,5523,...,3,58761330,45063584,0,0,4.0,1.0,2.0,3.0,2.0
4,75,1500933,"Дом по адресу Байкальская ул., д.18, к.2",124,1,2048798,18,2,148,5523,...,3,58761330,45063584,0,0,298.0,2.0,3.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,1066431,464022234,"Дом по адресу Тюменский пр., д.3, к.5",139,1,0,0,0,0,6022,...,3,58761330,45063584,0,0,115.0,222.0,255.0,7.0,5.0
0,1066438,464781953,"Дом по адресу Парковая 15-я ул., д.42, к.5",139,1,0,0,0,0,6022,...,3,58761330,45063584,0,0,108.0,87.0,174.0,7.0,15.0
1,1066437,464781953,"Дом по адресу Парковая 15-я ул., д.42, к.5",139,1,0,0,0,0,6022,...,3,58761330,45063584,0,0,108.0,89.0,174.0,7.0,15.0
2,1066439,464781953,"Дом по адресу Парковая 15-я ул., д.42, к.5",139,1,0,0,0,0,6022,...,3,58761330,45063584,0,0,312.0,104.0,209.0,7.0,5.0


In [28]:
for_model = sum_df.drop(['NAME', 'ID', 'index'], axis=1)
for_model.iloc[1000:].to_csv('for_model.csv', sep=',', index=False)
for_model.iloc[:1000].to_csv('for_test.csv', sep=',', index=False)

In [29]:
for_test = []

In [33]:
# sum_df = sum_df[sum_df['UNOM']]
for_predict = (sum_df.groupby("ID", as_index=False, group_keys=False).apply(lambda x: x.nlargest(1, "UNOM")))
for_predict.drop(['index','WORK_CLASS'], axis=1).to_csv('for_predict_without_unom.csv', sep=',', index=False)
for_predict

,index,ID,NAME,COL_756,COL_757,COL_758,COL_759,COL_760,COL_761,COL_762,...,COL_2463,COL_3163,COL_3243,COL_3363,COL_3468,WORK_NAME,CREATE_DATE,CLOSE_DATE,SOURCE,WORK_CLASS
272,212,1500933,"Дом по адресу Байкальская ул., д.18, к.2",124,1,2048798,18,2,148,5523,...,3,58761330,45063584,0,0,140.0,360.0,459.0,3.0,23.0
95,363,1500943,"Дом по адресу Перовская ул., д.48",74,1,2048912,4,2,20,584,...,3,58761330,45063584,0,0,166.0,355.0,360.0,1.0,18.0
84,438,1500951,"Дом по адресу Парковая 3-я ул., д.39, к.3",79,1,2048912,5,3,60,2181,...,3,58761330,45063584,0,0,300.0,364.0,365.0,0.0,16.0
484,838,1500954,"Дом по адресу Свободный просп., д.11, к.2",92,1,2048755,9,12,431,1796,...,3,58761330,45063584,0,0,68.0,362.0,458.0,6.0,12.0
236,1119,1500958,"Дом по адресу Кусковская ул., д.23, к.1",87,1,2048912,5,4,64,3588,...,3,58761330,45063584,0,0,104.0,363.0,460.0,3.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,1066418,451132136,"Дом по адресу Черное Озеро ул., д.4",139,1,0,0,0,0,6022,...,3,58761330,45063584,0,0,108.0,363.0,424.0,7.0,15.0
2,1066420,453685119,"Дом по адресу Тюменский пр., д.3, к.1",139,1,0,0,0,0,6022,...,3,58761330,45063584,0,0,115.0,233.0,258.0,7.0,5.0
0,1066427,463322754,"Дом по адресу Плеханова ул., д.18",139,1,0,0,0,0,6022,...,3,58761330,45063584,0,0,238.0,105.0,106.0,6.0,10.0
1,1066431,464022234,"Дом по адресу Тюменский пр., д.3, к.5",139,1,0,0,0,0,6022,...,3,58761330,45063584,0,0,115.0,222.0,255.0,7.0,5.0


In [31]:
# # сохраняем для обучения
# big_df.to_csv('/Users/sergeyesenin/PycharmProjects/hakaton2/backend/services/upload_data/src/servicedata_for_model2.csv', sep=',', index=False)


In [32]:
# # сохраняем для предикта
# big_df.to_csv('/Users/sergeyesenin/PycharmProjects/hakaton2/backend/services/upload_data/src/servicedata_for_model2.csv', sep=',', index=False)